In [1]:
import torch
import torch.nn as nn
import re
from transformers import AutoTokenizer
import pickle
import numpy as np
import os

2022-08-12 06:23:02.219214: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-12 06:23:02.219235: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [20]:
GBASE = "./reverse_graphs/"
endebase = "mt1n_en-de_bfs_recom_1_80_False_0.4_True_False_4_5_rcb_0.9_0.0_0.9/"
frenbase = "mtn1_fr-en_bfs_recom_1_-1_False_0.4_True_False_4_5_rcb_0.9_0.0_0.9/"

def load_graph(fname):
    return pickle.load(open(fname,'rb'))

def flatten_lattice(graph):
    tokdicts = []
    visited = []
    poslist = []
    greedy_flatten(tokdicts, visited, graph['root'], 0)
    return tokdicts
    
def greedy_flatten(tdicts, visited, node, pos):
    if node.uid in visited:
        print("cycle here")
        return
    node.pos = pos
    tdicts.append(node)
    visited.append(node.uid)
    
    scosort = list(np.argsort(node.next_scores))
    
    # TODO check which direction we need to go from argsort
    for i in range(0, len(scosort)):
        greedy_flatten(tdicts, visited, node.nextlist[scosort[i]], pos+1)
        
def get_processed_graph_data(lanbase, stop=-1):
    base = GBASE+lanbase
    paths = os.listdir(base)
    print(len(paths))
    result = []
    if stop==-1:
        stop = len(paths)
    for i in range(0, stop):
        curgraph = load_graph(base+paths[i])
        result.append(flatten_lattice(curgraph))
    return result


In [21]:
processedgraphs = get_processed_graph_data(frenbase, 1)

301
cycle here
cycle here
cycle here


In [19]:
def print_proctoks(revnodes):
    for rev in revnodes:
        print(rev.token_str, " - ", rev.pos)

print_proctoks(processedgraphs[0])

</s>  -  0
en_XX  -  1
On  -  2
Monday  -  3
,  -  4
  -  5
20,000  -  6
police  -  7
men  -  8
will  -  9
be  -  10
de  -  11
plo  -  12
y  -  13
ed  -  14
to  -  15
the  -  16
Cair  -  17
o  -  18
Police  -  19
Academy  -  20
,  -  21
where  -  22
Mohamed  -  23
Mor  -  24
si  -  25
will  -  26
be  -  27
tried  -  28
.  -  29
will  -  8
be  -  9
de  -  10
officer  -  8
s  -  9
will  -  10
be  -  11
de  -  12
plo  -  13
y  -  14
ed  -  15
before  -  16
the  -  17
Cair  -  18
o  -  19
to  -  16
the  -  17
Cair  -  18
o  -  19
Police  -  20
Academy  -  21
where  -  22
Mohamed  -  23
Mor  -  24
to  -  22
try  -  23
Mohamed  -  24
Mor  -  25
si  -  26
.  -  27
,  -  22
where  -  23
Mohamed  -  24
Mor  -  25
si  -  26
is  -  27
to  -  28
be  -  29
tried  -  30
.  -  31
will  -  27
be  -  28
tried  -  29
.  -  30


In [20]:
# data to work with
text = (
   'Hello, how are you? I am Romeo.\n'
   'Hello, Romeo My name is Juliet. Nice to meet you.\n'
   'Nice meet you too. How are you today?\n'
   'Great. My baseball team won the competition.\n'
   'Oh Congratulations, Juliet\n'
   'Thanks you Romeo'
)

# pre-processing text (all technically in one string)
sentences = re.sub("[.,!?\\-]", '', text.lower()).split('\n')  # filter '.', ',', '?', '!'
# split by spaces
word_list = list(set(" ".join(sentences).split()))

In [11]:
# adding special tokens
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}

# create vocabulary dictionaries (word->token id mapping)
for i, w in enumerate(word_list):
    word_dict[w] = i + 4
    number_dict = {i: w for i, w in enumerate(word_dict)}
    vocab_size = len(word_dict)

In [14]:
def make_batch():
    batch = []
    positive = negative = 0
    
    # keep things balanced? 
    while positive != batch_size/2 or negative != batch_size/2:
        # get 2 random sentences, we want to check if they're contiguous or not
        tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange(len(sentences)) 
        
        # not sure where token_list comes from 
        tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]

        # create a list with all the appropriate token ids for the input formatting
        input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']]

        # handle segment embedding?
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

        # MASK LM
        # get num of tokens to mask
        n_pred =  min(max_pred, max(1, int(round(len(input_ids) * 0.15)))) # 15 % of tokens in one sentence
        cand_maked_pos = [i for i, token in enumerate(input_ids)
                         if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]
        shuffle(cand_maked_pos)
        
        # randomly select 15% of tokens to mask
        masked_tokens, masked_pos = [], []
        for pos in cand_maked_pos[:n_pred]:
            
            masked_pos.append(pos)
            masked_tokens.append(input_ids[pos])
            # usually just mask
            if random() < 0.8:  # 80%
                input_ids[pos] = word_dict['[MASK]'] # make mask
            # sometimes just put a random word in
            elif random() < 0.5:  # 10%
                index = randint(0, vocab_size - 1) # random index in vocabulary
                input_ids[pos] = word_dict[number_dict[index]] # replace

        # Zero Paddings
        n_pad = maxlen - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        # Zero Padding (100% - 15%) tokens
        if max_pred > n_pred:
            n_pad = max_pred - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        if tokens_a_index + 1 == tokens_b_index and positive < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # IsNext
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NotNext
            negative += 1
    return batch

In [23]:
# print(torch.arange(30, dtype=torch.long).expand_as(input_ids))
class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
        self.pos_embed = nn.Embedding(maxlen, d_model)  # position embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long)
        pos = pos.unsqueeze(0).expand_as(x)  # (seq_len,) -> (batch_size, seq_len)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

In [24]:
def get_attn_pad_mask(seq_q, seq_k):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

In [ ]:
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.embedding = Embedding()
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ1 = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.activ2 = gelu
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        # decoder is shared with embedding layer
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_mode, d_model]
        # it will be decided by first token(CLS)
        h_pooled = self.activ1(self.fc(output[:, 0])) # [batch_size, d_model]
        logits_clsf = self.classifier(h_pooled) # [batch_size, 2]

        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) # [batch_size, max_pred, d_model]

        # get masked position from final output of transformer.
        h_masked = torch.gather(output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
        h_masked = self.norm(self.activ2(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias # [batch_size, max_pred, n_vocab]

        return logits_lm, logits_clsf